In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

In [51]:
df = pd.read_csv('data/samples_updated.csv')
df

,filename,subject,group,race,age,x0,y0,x1,y1,x2,...,x63,y63,x64,y64,x65,y65,x66,y66,x67,y67
0,E:\Lurie Research\final cohort3\CONTROL\Africa...,1452,CONTROL,African American,0-1,62,313,80,367,98,...,324,460,367,460,327,467,314,471,300,472
1,E:\Lurie Research\final cohort3\CONTROL\Africa...,1453,CONTROL,African American,0-1,61,115,63,131,67,...,130,175,142,179,130,177,126,177,122,176
2,E:\Lurie Research\final cohort3\CONTROL\Africa...,1454,CONTROL,African American,0-1,1391,616,1393,679,1396,...,1662,829,1689,846,1661,847,1644,849,1627,846
3,E:\Lurie Research\final cohort3\CONTROL\Africa...,1455,CONTROL,African American,0-1,361,117,340,160,322,...,457,389,503,444,447,404,430,387,418,368
4,E:\Lurie Research\final cohort3\CONTROL\Africa...,1456,CONTROL,African American,0-1,91,116,88,149,86,...,209,279,224,284,207,274,199,272,191,269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,E:\Lurie Research\final cohort3\CCHS\Other\18-...,98,CCHS,Other,18-25,1473,801,1486,891,1502,...,1889,1017,1961,1040,1889,1022,1854,1026,1821,1026
2885,E:\Lurie Research\final cohort3\CCHS\Other\18-...,98,CCHS,Other,18-25,1496,1134,1506,1223,1522,...,1908,1352,1980,1374,1909,1352,1874,1356,1842,1357
2886,E:\Lurie Research\final cohort3\CCHS\Other\26-...,108,CCHS,Other,26-35,131,207,134,237,137,...,272,301,299,297,272,304,260,306,248,305
2887,E:\Lurie Research\final cohort3\CCHS\Other\26-...,108,CCHS,Other,26-35,1130,491,1130,555,1131,...,1405,680,1471,686,1404,680,1382,681,1359,679


In [54]:
duplicate_subject = df[df.duplicated('subject')]
duplicate_subject

,filename,subject,group,race,age,x0,y0,x1,y1,x2,...,x63,y63,x64,y64,x65,y65,x66,y66,x67,y67
36,E:\Lurie Research\final cohort3\CONTROL\Africa...,1473,CONTROL,African American,2-3,50,133,53,150,56,...,138,166,148,178,137,183,130,183,123,182
38,E:\Lurie Research\final cohort3\CONTROL\Africa...,1485,CONTROL,African American,2-3,48,127,50,143,53,...,133,171,150,172,133,172,129,172,123,172
70,E:\Lurie Research\final cohort3\CONTROL\Africa...,1486,CONTROL,African American,4-5,54,121,55,139,58,...,137,176,146,180,137,182,131,183,125,183
72,E:\Lurie Research\final cohort3\CONTROL\Africa...,1473,CONTROL,African American,4-5,51,125,52,144,56,...,139,168,151,176,139,183,131,184,123,184
89,E:\Lurie Research\final cohort3\CONTROL\Africa...,1453,CONTROL,African American,6-8,65,115,68,131,71,...,130,177,144,179,130,179,125,178,122,177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2883,E:\Lurie Research\final cohort3\CCHS\Other\15-...,105,CCHS,Other,15-17,1353,893,1364,1031,1384,...,2017,1287,2138,1297,2018,1288,1952,1295,1883,1287
2884,E:\Lurie Research\final cohort3\CCHS\Other\18-...,98,CCHS,Other,18-25,1473,801,1486,891,1502,...,1889,1017,1961,1040,1889,1022,1854,1026,1821,1026
2885,E:\Lurie Research\final cohort3\CCHS\Other\18-...,98,CCHS,Other,18-25,1496,1134,1506,1223,1522,...,1908,1352,1980,1374,1909,1352,1874,1356,1842,1357
2887,E:\Lurie Research\final cohort3\CCHS\Other\26-...,108,CCHS,Other,26-35,1130,491,1130,555,1131,...,1405,680,1471,686,1404,680,1382,681,1359,679


In [68]:
# get row #s of duplicated subjects
duplicated_rows = {}
subjects = []
# for subject in duplicate_subject['subject']:
#     
for idx, row in duplicate_subject.iterrows():
    subject = row['subject']
    if subject not in duplicated_rows.keys(): # add key
        duplicated_rows[subject] = [idx]
    else:
        duplicated_rows[subject].append(idx)

36 1473
38 1485
70 1486
72 1473
89 1453
90 1486
91 1473
92 1482
93 1488
94 1485
135 1487
136 1524
137 1485
152 1486
154 1485
155 1453
156 1487
157 1524
158 1473
159 1488
160 1488
164 136
170 158
177 167
191 200
329 238
391 152
392 157
393 158
395 161
396 161
397 162
398 164
400 167
401 171
402 172
404 174
405 177
407 179
412 185
414 187
415 187
417 190
418 191
421 194
422 196
425 198
426 199
427 200
428 202
593 599
627 122
628 130
630 132
632 137
634 140
637 150
638 151
639 151
642 159
643 159
644 163
645 164
646 165
648 172
649 173
651 176
652 177
653 179
654 180
655 181
657 184
658 185
659 186
660 187
662 189
663 190
665 199
666 199
667 200
668 201
669 202
798 129
799 131
800 133
801 136
802 137
803 137
804 142
805 144
808 147
809 147
810 151
812 157
813 158
814 158
815 160
816 161
817 163
819 167
820 170
821 170
822 171
823 172
824 174
826 176
827 176
828 177
829 178
830 180
831 181
832 181
833 181
834 182
835 186
836 186
837 186
838 187
839 187
840 187
841 190
842 191
843 194
844 1

In [47]:
X = torch.from_numpy(np.load('data/stratify_people/samples.npy'))
y = torch.from_numpy(np.load('data/stratify_people/labels.npy'))

df_X = pd.DataFrame(X)
df_X = df_X.rename(columns={0: 'subject'})
df_y = pd.DataFrame(y)
df_y = df_y.rename(columns={0: 'label'})
df_X

,subject,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,tensor(1452.),tensor(1.),tensor(0.),tensor(0.),tensor(0.2931),tensor(0.0388),tensor(0.4095),tensor(0.0776),tensor(0.5194),tensor(0.1207),...,tensor(0.5647),tensor(0.6099),tensor(0.6573),tensor(0.6099),tensor(0.5711),tensor(0.6250),tensor(0.5431),tensor(0.6336),tensor(0.5129),tensor(0.6358)
1,tensor(1453.),tensor(1.),tensor(0.),tensor(0.),tensor(0.1408),tensor(0.0141),tensor(0.2535),tensor(0.0423),tensor(0.3662),tensor(0.0704),...,tensor(0.4859),tensor(0.5634),tensor(0.5704),tensor(0.5915),tensor(0.4859),tensor(0.5775),tensor(0.4577),tensor(0.5775),tensor(0.4296),tensor(0.5704)
2,tensor(1454.),tensor(1.),tensor(0.),tensor(0.),tensor(0.1631),tensor(0.0043),tensor(0.2983),tensor(0.0107),tensor(0.4335),tensor(0.0279),...,tensor(0.5815),tensor(0.6202),tensor(0.6395),tensor(0.6567),tensor(0.5794),tensor(0.6588),tensor(0.5429),tensor(0.6631),tensor(0.5064),tensor(0.6567)
3,tensor(1455.),tensor(1.),tensor(0.),tensor(0.1272),tensor(0.1422),tensor(0.0819),tensor(0.2349),tensor(0.0431),tensor(0.3319),tensor(0.0129),...,tensor(0.3341),tensor(0.7284),tensor(0.4332),tensor(0.8470),tensor(0.3125),tensor(0.7608),tensor(0.2759),tensor(0.7241),tensor(0.2500),tensor(0.6832)
4,tensor(1456.),tensor(1.),tensor(0.),tensor(0.0244),tensor(0.0325),tensor(0.0122),tensor(0.1667),tensor(0.0041),tensor(0.3049),tensor(0.),...,tensor(0.5041),tensor(0.6951),tensor(0.5650),tensor(0.7154),tensor(0.4959),tensor(0.6748),tensor(0.4634),tensor(0.6667),tensor(0.4309),tensor(0.6545)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,tensor(98.),tensor(0.),tensor(7.),tensor(0.),tensor(0.3119),tensor(0.0178),tensor(0.4350),tensor(0.0397),tensor(0.5540),tensor(0.0643),...,tensor(0.5691),tensor(0.6074),tensor(0.6676),tensor(0.6389),tensor(0.5691),tensor(0.6142),tensor(0.5212),tensor(0.6197),tensor(0.4761),tensor(0.6197)
2885,tensor(98.),tensor(0.),tensor(7.),tensor(0.),tensor(0.3182),tensor(0.0138),tensor(0.4408),tensor(0.0358),tensor(0.5606),tensor(0.0620),...,tensor(0.5675),tensor(0.6185),tensor(0.6667),tensor(0.6488),tensor(0.5689),tensor(0.6185),tensor(0.5207),tensor(0.6240),tensor(0.4766),tensor(0.6253)
2886,tensor(108.),tensor(0.),tensor(8.),tensor(0.),tensor(0.2407),tensor(0.0124),tensor(0.3651),tensor(0.0249),tensor(0.4855),tensor(0.0456),...,tensor(0.5851),tensor(0.6307),tensor(0.6971),tensor(0.6141),tensor(0.5851),tensor(0.6432),tensor(0.5353),tensor(0.6515),tensor(0.4855),tensor(0.6473)
2887,tensor(108.),tensor(0.),tensor(8.),tensor(0.),tensor(0.3092),tensor(0.),tensor(0.4313),tensor(0.0019),tensor(0.5496),tensor(0.0153),...,tensor(0.5248),tensor(0.6698),tensor(0.6508),tensor(0.6813),tensor(0.5229),tensor(0.6698),tensor(0.4809),tensor(0.6718),tensor(0.4370),tensor(0.6679)


In [48]:
duplicate_subject = df_X[df_X.duplicated('subject')]
duplicate_subject

,subject,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
1720,tensor(193.),tensor(-1.),tensor(7.),tensor(0.),tensor(0.2360),tensor(0.0200),tensor(0.3520),tensor(0.0560),tensor(0.4640),tensor(0.0960),...,tensor(0.5840),tensor(0.6360),tensor(0.6880),tensor(0.6160),tensor(0.5880),tensor(0.6360),tensor(0.5440),tensor(0.6480),tensor(0.5000),tensor(0.6480)
1816,tensor(163.),tensor(-1.),tensor(8.),tensor(0.),tensor(0.2266),tensor(0.0121),tensor(0.3565),tensor(0.0302),tensor(0.4804),tensor(0.0514),...,tensor(0.5136),tensor(0.6224),tensor(0.6224),tensor(0.6103),tensor(0.5136),tensor(0.6465),tensor(0.4532),tensor(0.6586),tensor(0.4018),tensor(0.6495)
2380,tensor(6.),tensor(-1.),tensor(2.),tensor(0.),tensor(0.0395),tensor(0.0115),tensor(0.1646),tensor(0.0289),tensor(0.2839),tensor(0.0481),...,tensor(0.4986),tensor(0.5332),tensor(0.5900),tensor(0.5438),tensor(0.4986),tensor(0.5284),tensor(0.4610),tensor(0.5322),tensor(0.4244),tensor(0.5236)
2442,tensor(3.),tensor(-1.),tensor(3.),tensor(0.),tensor(0.1691),tensor(0.0196),tensor(0.2941),tensor(0.0392),tensor(0.4105),tensor(0.0600),...,tensor(0.5257),tensor(0.5956),tensor(0.6225),tensor(0.6127),tensor(0.5257),tensor(0.6042),tensor(0.4853),tensor(0.6054),tensor(0.4449),tensor(0.6017)
2655,tensor(40.),tensor(-1.),tensor(6.),tensor(0.),tensor(0.2577),tensor(0.0052),tensor(0.3763),tensor(0.0193),tensor(0.4897),tensor(0.0374),...,tensor(0.5000),tensor(0.6070),tensor(0.6224),tensor(0.6340),tensor(0.4987),tensor(0.6044),tensor(0.4497),tensor(0.6044),tensor(0.3969),tensor(0.5992)
2666,tensor(52.),tensor(-1.),tensor(6.),tensor(0.),tensor(0.2247),tensor(0.0165),tensor(0.3526),tensor(0.0454),tensor(0.4742),tensor(0.0680),...,tensor(0.5299),tensor(0.6082),tensor(0.6371),tensor(0.6330),tensor(0.5278),tensor(0.6041),tensor(0.4825),tensor(0.6041),tensor(0.4412),tensor(0.6021)
